In [ ]:
 train_path = "/content/drive/MyDrive/PlantVillage/train"
test_path = "/content/drive/MyDrive/PlantVillage/test"
val_path = "/content/drive/MyDrive/PlantVillage/validation"

In [ ]:
from os import listdir

In [ ]:
from keras.preprocessing.image import img_to_array
import cv2
def convert_image(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, tuple((256, 256)))   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [ ]:
images=[]
labels=[]
for folder in listdir(train_path):
  c=0
  for im in listdir(train_path+'/'+str(folder)):
    c+=1
    images.append(convert_image(train_path+'/'+str(folder)+'/'+im))
    labels.append(int(folder))
    if c==300:
      break

In [ ]:
len(images)

2998

In [ ]:
len(labels)


2998

In [ ]:
images[0].shape

(256, 256, 3)

In [ ]:
import numpy as np
images= np.asarray(images)

In [ ]:
labels=np.asarray(labels)

In [ ]:
images=images/255

In [ ]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test=train_test_split(images,labels,test_size=0.2)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
dat = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [ ]:
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
model = Sequential()
inputShape = (256,256,3)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

In [ ]:
EPOCHS = 25
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
#directory_root = '../input/plantvillage/'
width=256
height=256
depth=3

In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / 25)
# distribution
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
cnn_checkpoint = ModelCheckpoint(filepath='custom_model_best_5epochs.hdf5',
                                  save_best_only=True,
                                  monitor='val_accuracy',
                                  mode='max',
                                  verbose=2)
history = model.fit_generator(
    dat.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=11, verbose=1,
    callbacks=[cnn_checkpoint]
    )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if sys.path[0] == '':


Epoch 1/11
74/74 [==============================] - ETA: 0s - loss: 1.6513 - accuracy: 0.5194
Epoch 1: val_accuracy improved from -inf to 0.09167, saving model to custom_model_best_5epochs.hdf5
74/74 [==============================] - 557s 8s/step - loss: 1.6513 - accuracy: 0.5194 - val_loss: 7.6878 - val_accuracy: 0.0917
Epoch 2/11
74/74 [==============================] - ETA: 0s - loss: 1.2914 - accuracy: 0.6002
Epoch 2: val_accuracy improved from 0.09167 to 0.09500, saving model to custom_model_best_5epochs.hdf5
74/74 [==============================] - 554s 7s/step - loss: 1.2914 - accuracy: 0.6002 - val_loss: 26.3440 - val_accuracy: 0.0950
Epoch 3/11
74/74 [==============================] - ETA: 0s - loss: 1.3355 - accuracy: 0.6074
Epoch 3: val_accuracy improved from 0.09500 to 0.17167, saving model to custom_model_best_5epochs.hdf5
74/74 [==============================] - 553s 7s/step - loss: 1.3355 - accuracy: 0.6074 - val_loss: 9.2778 - val_accuracy: 0.1717
Epoch 4/11
74/74 [===

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['Train_accuracy','val_accuracy'],loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Train_loss','val_loss'],loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
pred=custom_model.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test,np.argmax(pred,axis = 1))

sns.heatmap(cm,annot = True,fmt = '.2f')